Binary classification (basic/advanced) for synsets

We use the already classified synsets from data folder as training/test set.
For each synset provided there are one or more relevant words that belongs to the synset. And there is the label (basic/advanced) for the synset.

We then consider as features for classification the following:
1. The depth of the synset in the WordNet hierarchy
2. The pronunciation complexity of the first word **in the dataset**
3. The length of the first word **in the dataset**
4. The synset classification to concrete or abstract concept

Given a new synset we want to classify we then get its features by:
1. Getting the depth of the sysnet in the WordNet hierarchy
2. Getting the pronunciation complexity of the first (most frequently used) word **in the synset**
3. Getting the length of the first (most frequently used) word **in the synset**
4. Predicting the synset classification to concrete or abstract concept

After defining data we train a binary classifier to predict the label (basic/advanced) of the synset.
Since we have a small dataset we use a simple logistic regression classifier trained using 5-fold cross validation.

Steps:
1. Load and format the JSON dataset (synsets, word(s), labels, definitions) 

In [1]:
from typing import List, Dict
import json
import pandas as pd
from pandas import DataFrame
import openpyxl # install it as it is required by pandas to read excel files

import nltk
from nltk.corpus.reader import Synset
from nltk.corpus import wordnet as wn
from nltk.corpus import cmudict

import spacy
import joblib

# nltk.download('wordnet')
# # download the CMU Pronouncing Dictionary
# nltk.download('cmudict')
# spacy.cli.download("en_core_web_md")

nlp = spacy.load("en_core_web_md")

## 1. Load and format the JSON dataset (synsets, word(s), labels, definitions)

In [2]:
with open('data/1.json') as f:
    data = json.load(f)
    dataset = data['dataset']
    labels = data['answers']

Get Synset from string of the form "Synset('word.pos.n')"

In [3]:
def get_synset_from_string(s: str) -> Synset:
    # find first ' and last '
    start = s.find('\'')
    end = s.rfind('\'')
    synset_name = s[start + 1:end]
    return wn.synset(synset_name)

Calculate pronunce complexity based on number of phonemes
If word not found in dictionary return 50, indicating high complexity (max complexity is around 25 in this dataset)

In [4]:
pronunce_dict = cmudict.dict()

def calculate_pronunce_complexity(sentence: str) -> int:
    sentence = sentence.split()
    complexity = 0
    for word in sentence:
        if word.lower() in pronunce_dict:
            phonemes = pronunce_dict[word.lower()][0]  # get phonetic representation
            complexity += len(phonemes)  # complexity based on number of phonemes
        else: # TODO check if returning high complexity if one word not found is better. Maybe
            return 50
    
    if complexity == 0: # if word not found in dictionary return high complexity
        return 50
    return complexity

Select synset and word(s) from dataset

In [5]:
# get classifier with joblib
concrete_abstract_cls = joblib.load('trained_models/concrete_abstract_classifier.joblib')

In [6]:
# dataframe containing synset, word(s), definition, label
cols = ['synset', 'words', 'synset_depth', 'pronunce_complexity', 'first_word_length', 'abstract', 'label', 'definition']
dataset_df: DataFrame = pd.DataFrame(columns=cols)

splitted_list: List[List[str]] = []
label_index = 0
for row in dataset:
    row_list = []
    temp_split = row.split(':')
    for elem in temp_split:
        splitted = elem.split('|')
        row_list.extend([x for x in splitted])
    
    # get synset
    synset = get_synset_from_string(row_list[0])
    # get words
    words = row_list[1]
    words = words.split(',')
    words = [word.strip() for word in words]
    # take only first word for now, it should be the most frequently used
    first_word = words[0]
    words = ",".join(words)
    # get synset depth
    synset_depth = synset.max_depth()
    # get pronunce complexity
    pronunce = calculate_pronunce_complexity(first_word)
    # get first word length
    first_word_length = len(first_word)
    # get concreteness
    word_vector = list(nlp(first_word))[0].vector
    is_abstract = concrete_abstract_cls.predict([word_vector])[0]
    # get label
    label = labels[label_index]
    # get definition
    definition = row_list[3]
    # add row to dataframe
    new_row = [[synset, first_word, synset_depth, pronunce, first_word_length, is_abstract, label, definition]]
    dataset_df = pd.concat(
        [dataset_df, pd.DataFrame(new_row, columns=cols)], 
        ignore_index=True)
    label_index += 1

In [7]:
dataset_df.head(600)

,synset,words,synset_depth,pronunce_complexity,first_word_length,abstract,label,definition
0,Synset('war.n.01'),war,7,3,3,1,basic,the waging of armed conflict against an enemy
1,Synset('fiefdom.n.01'),fiefdom,6,6,7,1,advanced,the domain controlled by a feudal lord
2,Synset('bed.n.03'),bed,5,3,3,0,basic,a depression forming the ground under a body ...
3,Synset('return_on_invested_capital.n.01'),return on invested capital,6,22,26,1,advanced,"(corporate finance) the amount, expressed as ..."
4,Synset('texture.n.02'),texture,9,6,7,1,basic,the essential quality of something
...,...,...,...,...,...,...,...,...
499,Synset('reading.n.03'),reading,6,5,7,1,basic,a datum about some physical state that is pre...
500,Synset('sanctimoniousness.n.01'),sanctimoniousness,10,50,17,1,advanced,the quality of being hypocritically devout
501,Synset('chalcedony.n.01'),chalcedony,8,9,10,0,advanced,a milky or greyish translucent to transparent...
502,Synset('stopcock.n.01'),stopcock,11,50,8,1,advanced,faucet consisting of a rotating device for re...


In [8]:
# count number of abstract advanced words and concrete basic words
abstract_advanced_count = dataset_df[(dataset_df['abstract'] == 1) & (dataset_df['label'] == "advanced")].shape[0]
concrete_basic_count = dataset_df[(dataset_df['abstract'] == 0) & (dataset_df['label'] == "basic")].shape[0]

print(f"Number of abstract advanced words: {abstract_advanced_count}")
print(f"Number of concrete basic words: {concrete_basic_count}")

Number of abstract advanced words: 134
Number of concrete basic words: 117


Measure correlation between features and label (basic/advanced)

In [ ]:
from scipy.stats import chi2_contingency

def chi_square_test(feature, label):
    contingency_table = pd.crosstab(feature, label)
    chi2, p_val, _, _ = chi2_contingency(contingency_table)
    return chi2, p_val

In [ ]:
chi_square_results = {}
for col in dataset_df.columns:
    if col not in ['synset', 'words', 'label', 'definition']:
        continue
    pearson_corr = dataset_df[col].corr(dataset_df['label'])
    spearman_corr = dataset_df[col].corr(dataset_df['label'], 'spearman')
    chi2, p_val = chi_square_test(dataset_df[col], dataset_df['label'])
    chi_square_results[col] = {'Chi-square': chi2, 'p-value': p_val}
    print(f"Pearson correlation between {col} and label: {pearson_corr}")
    print(f"Spearman correlation between {col} and label: {spearman_corr}")